<a href="https://colab.research.google.com/github/MrPer4ik/PatternRecognition/blob/main/AdditionalTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, BaggingClassifier
from sklearn import metrics

Кожен запис є прикладом "руки" в покері, що складається з п’яти гральних карт, витягнутих зі стандартної колоди 52. Кожна карта описується двома атрибутами (масті та рангу), загалом 10. Існує один атрибут ***CLASS***, який описує силу (можливість виграшу) покерної "руки". Порядок карт важливий, тому існує 480 можливих роздач **Роял Флеш** порівняно з 4 (по одній для кожної масті).

<h2>Інформація про атрибути:</h2>

1) ***S1*** "Масть картки № 1" Порядковий (1-4), що представляє *{чирви, піки, бубни, трефи}*

2) ***C1*** "Ранг картки № 1" Числовий (1-13), що представляє (туз, 2, 3, ..., королева, король)

3) ***S2*** "Масть картки № 2" Порядковий (1-4), що представляє *{чирви, піки, бубни, трефи}*

4) ***C2*** "Ранг картки № 2" Числовий (1-13), що представляє (туз, 2, 3, ..., королева, король)

5) ***S3*** "Масть картки № 3" Порядок (1-4), що представляє *{чирви, піки, бубни, трефи}*

6) ***C3*** "Ранг картки № 3" Числовий (1-13), що представляє (туз, 2, 3, ..., королева, король)

7) ***S4*** "Масть картки № 4" Порядок (1-4), що представляє *{чирви, піки, бубни, трефи}*

8) ***C4*** "Ранг картки № 4" Числовий (1-13), що представляє (туз, 2, 3, ..., королева, король)

9) ***S5*** "Масть картки #5" Порядковий (1-4), що представляє *{чирви, піки, бубни, трефи}*

10) ***C5*** "Ранг картки 5" Числовий (1-13), що представляє (Туз, 2, 3, ..., Королева, Король)

11) ***CLASS*** Класифікація покерної "руки" Порядковий (0-9)

<h2>Можливі значення атрибуту CLASS</h2>

***0: Нічого в руках***; не визнана покерна рука

***1: Одна пара***; одна пара рівних рангів у межах п’яти карт

***2: Дві пари***; дві пари рівних рангів у межах п’яти карт

***3: Сет***; три рівні ранги в межах п’яти карт

***4: Стріт***; п'ять карт, послідовно ранжованих без пропусків

***5: Флеш***; п'ять карт однієї масті

***6: Фул гауз***; пара + сет

***7: Каре***; чотири рівні ранги в межах п’яти карт

***8: Стріт Флеш***; стріт + флеш

***9: Роял Флеш***; {Туз, Король, Королева, Валет, Десять} + флеш

# Підготовка та обробка даних

In [7]:
data = pd.read_csv('https://raw.githubusercontent.com/MrPer4ik/PatternRecognition/main/poker.csv')
data.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,CLASS
0,1,1,1,13,2,4,2,3,1,12,0
1,3,12,3,2,3,11,4,5,2,5,1
2,1,9,4,6,1,4,3,2,3,9,1
3,1,4,3,13,2,13,2,1,3,6,1
4,3,10,2,7,1,2,2,11,4,9,0


Сортуємо ранги карт за зростанням.

In [8]:
df = data.copy()
dfc = df[['C1', 'C2', 'C3', 'C4', 'C5']]
dfc.values.sort()
df[['C1', 'C2', 'C3', 'C4', 'C5']] = dfc
df = df[['C1', 'C2', 'C3', 'C4', 'C5', 'S1', 'S2', 'S3', 'S4', 'S5', 'CLASS']]
df.head()

,C1,C2,C3,C4,C5,S1,S2,S3,S4,S5,CLASS
0,1,3,4,12,13,1,1,2,2,1,0
1,2,5,5,11,12,3,3,3,4,2,1
2,2,4,6,9,9,1,4,1,3,3,1
3,1,4,6,13,13,1,3,2,2,3,1
4,2,7,9,10,11,3,2,1,2,4,0


Додаємо ознаки кількості відповідних рангів та мастей карт у "руці".

In [9]:
tmp = df[['C1', 'C2', 'C3', 'C4', 'C5']]
df['Cnt_C1'] = tmp.apply(lambda x: sum(x==x[0]) ,axis=1)
df['Cnt_C2'] = tmp.apply(lambda x: sum(x==x[1]) ,axis=1)
df['Cnt_C3'] = tmp.apply(lambda x: sum(x==x[2]) ,axis=1)
df['Cnt_C4'] = tmp.apply(lambda x: sum(x==x[3]) ,axis=1)
df['Cnt_C5'] = tmp.apply(lambda x: sum(x==x[4]) ,axis=1)

tmp = df[['S1', 'S2', 'S3', 'S4', 'S5']]
df['Cnt_S1'] = tmp.apply(lambda x: sum(x==x[0]) ,axis=1)
df['Cnt_S2'] = tmp.apply(lambda x: sum(x==x[1]) ,axis=1)
df['Cnt_S3'] = tmp.apply(lambda x: sum(x==x[2]) ,axis=1)
df['Cnt_S4'] = tmp.apply(lambda x: sum(x==x[3]) ,axis=1)    
df['Cnt_S5'] = tmp.apply(lambda x: sum(x==x[4]) ,axis=1)

df.head()

,C1,C2,C3,C4,C5,S1,S2,S3,S4,S5,CLASS,Cnt_C1,Cnt_C2,Cnt_C3,Cnt_C4,Cnt_C5,Cnt_S1,Cnt_S2,Cnt_S3,Cnt_S4,Cnt_S5
0,1,3,4,12,13,1,1,2,2,1,0,1,1,1,1,1,3,3,2,2,3
1,2,5,5,11,12,3,3,3,4,2,1,1,2,2,1,1,3,3,3,1,1
2,2,4,6,9,9,1,4,1,3,3,1,1,1,1,2,2,2,1,2,2,2
3,1,4,6,13,13,1,3,2,2,3,1,1,1,1,2,2,1,2,2,2,2
4,2,7,9,10,11,3,2,1,2,4,0,1,1,1,1,1,1,2,1,2,1


Додамо також ознаки, які показують різницю між сусідніми картами у "руці" (для визначення стріту), а також обчислимо кількість унікальних мастей у "руці" (для визначення флешу).

In [10]:
df['Diff1'] = df['C5'] - df['C4']
df['Diff2'] = df['C4'] - df['C3']
df['Diff3'] = df['C3'] - df['C2']
df['Diff4'] = df['C2'] - df['C1']

df['UniqueS'] = tmp.apply(lambda x: len(np.unique(x)) , axis=1)

df.head()

,C1,C2,C3,C4,C5,S1,S2,S3,S4,S5,CLASS,Cnt_C1,Cnt_C2,Cnt_C3,Cnt_C4,Cnt_C5,Cnt_S1,Cnt_S2,Cnt_S3,Cnt_S4,Cnt_S5,Diff1,Diff2,Diff3,Diff4,UniqueS
0,1,3,4,12,13,1,1,2,2,1,0,1,1,1,1,1,3,3,2,2,3,1,8,1,2,2
1,2,5,5,11,12,3,3,3,4,2,1,1,2,2,1,1,3,3,3,1,1,1,6,0,3,3
2,2,4,6,9,9,1,4,1,3,3,1,1,1,1,2,2,2,1,2,2,2,0,3,2,2,3
3,1,4,6,13,13,1,3,2,2,3,1,1,1,1,2,2,1,2,2,2,2,0,7,2,3,3
4,2,7,9,10,11,3,2,1,2,4,0,1,1,1,1,1,1,2,1,2,1,1,1,2,5,4


In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], 
                                                    df.iloc[:, -1], 
                                                    test_size=0.3, 
                                                    stratify=df.iloc[:, -1])

In [18]:
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

# Навчання базових моделей

In [13]:
models = {
    'Linear SVM': SVC(kernel='linear'),
    'RBF SVM': SVC(),
    'KNeighbors': KNeighborsClassifier(n_neighbors=50, n_jobs=-1),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(), 
    'Logistic Regression': LogisticRegression(n_jobs=-1)
}

In [21]:
for model in models:
    clf = models[model]
    train_score = np.mean(evaluate_model(clf, X_train, y_train))
    test_score = np.mean(evaluate_model(clf, X_test, y_test))
    print(f'{model}\nTrain score: {train_score}\nTest score: {test_score}\n\n')

Linear SVM
Train score: 1.0
Test score: 1.0


RBF SVM
Train score: 1.0
Test score: 1.0


KNeighbors
Train score: 0.7652551970009323
Test score: 0.7043333333333333


Naive Bayes
Train score: 0.9258407146599172
Test score: 0.9298666666666666


Decision Tree
Train score: 1.0
Test score: 1.0


Logistic Regression
Train score: 0.9823328795270072
Test score: 0.9815333333333333




# Стекінг

Об'єднаємо у стекінг класифікатор окремо моделі, які дали відмінну якість, та ті, які показали себе не з найкращої сторони при вирішенні даної задачі, та порівняємо отримані результати.

In [28]:
stack_models = {
    'Stack from better models': StackingClassifier(
        estimators=[('svm', models['Linear SVM']),
                    ('dt', models['Decision Tree'])],
        final_estimator=models['Logistic Regression'], cv=10, n_jobs=-1),
    'Stack from worse models': StackingClassifier(
        estimators=[('kn', models['KNeighbors']),
                    ('nb', models['Naive Bayes'])],
        final_estimator=models['Logistic Regression'], cv=10, n_jobs=-1) 
}

In [29]:
for model in stack_models:
    clf = stack_models[model]
    train_score = np.mean(evaluate_model(clf, X_train, y_train))
    test_score = np.mean(evaluate_model(clf, X_test, y_test))
    print(f'{model}\nTrain score: {train_score}\nTest score: {test_score}\n\n')

Stack from better models
Train score: 1.0
Test score: 1.0


Stack from worse models
Train score: 0.9656704719721866
Test score: 0.9538111111111113




Отже, можна зробити висновок, що, по-перше, для "хороших" моделей використання ансамблей не має жодного сенсу, адже відмінна якість вже досягнута, а по-друге, якість для не настільки хороших моделей значно зростає. Наприклад, моделі К найближчих сусідів та наївного Байєса мали точність 70 та 92 відсотки на тестовій вибірці відповідно. А при їх стекінгу та застосуванні наприкінці логістичної регресії підвищує якість до 95%.

# Бегінг

In [32]:
bag_models = {
    'Bag Decision Tree': BaggingClassifier(
        base_estimator=models['Decision Tree'], n_estimators=10),
    'Bag Naive Bayes': BaggingClassifier(
        base_estimator=models['Naive Bayes'], n_estimators=10) 
}

In [33]:
for model in bag_models:
    clf = bag_models[model]
    train_score = np.mean(evaluate_model(clf, X_train, y_train))
    test_score = np.mean(evaluate_model(clf, X_test, y_test))
    print(f'{model}\nTrain score: {train_score}\nTest score: {test_score}\n\n')

Bag Decision Tree
Train score: 1.0
Test score: 1.0


Bag Naive Bayes
Train score: 0.9258216602371768
Test score: 0.9295222222222221




Для бегінгу ситуація схожа, але можна також помітити, що якість для наївного байєсу не сильно змінилася, а для дерева рішень так і залишилася ідеальною. Треба також зауважити, що бегінг працює значно швидше за стекінг, бо не вимагає навчання різних моделей (це також залежить і від базових моделей, наприклад, у нашому випадку "bottleneck"-ом був саме алгоритм К сусідів).